# RECORD LINKAGE AND MEASUREMENT
By SiqiWang

# Check point 1: (Assignment 3 due March 10th)  <a class="anchor" id="checkpoint1"></a>
Add a column to the `DF_2015` dataframe that has the total value of New National Cancer Instute Projects (3points)
Due 

In [41]:
# Calculate the sum of the total new NCI costs for each PI
NCI_new_cost = NCI_new.groupby('PI_NAME')['TOTAL_COST'].sum().sort_values(ascending=False)

# Convert into a dataframe and reset index
NCI_new_cost = NCI_new_cost.to_frame().reset_index()
NCI_new_cost

# Rename the colums NCI_New_TOTAL_COST
NCI_new_cost.rename(columns={'TOTAL_COST':'NCI_New_Total_Cost_2015'}, inplace=True)
NCI_new_cost

# Outer merge DF2015 and NCI_new_cost (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html)

DF_2015 = pd.merge(NCI_new_cost,DF_2015, on='PI_NAME', how = 'outer')

DF_2015.head(15)

,PI_NAME,NCI_New_Total_Cost_2015,Total_Projects_2015,Total_Cost_All_2015,Total_NCI_Projects_2015,NCI_Total_Cost_2015,Total_New_NCI_Projects_2015
0,"ZELENIUCH-JAQUOTTE, ANNE",788279.0,1,788279.0,1.0,788279.0,1.0
1,"KIRCHHOFF, TOMAS",478926.0,2,663257.0,2.0,663257.0,1.0
2,"AIFANTIS, IANNIS",457533.0,4,1535056.0,4.0,1535056.0,1.0
3,"CARUCCI, JOHN A",444422.0,1,444422.0,1.0,444422.0,1.0
4,"GOLD, HEATHER TAFFET",424833.0,1,424833.0,1.0,424833.0,1.0
5,"SCHNEIDER, ROBERT",419016.0,1,419016.0,1.0,419016.0,1.0
6,"ROTHENBERG, ELI",253893.0,2,580181.0,1.0,253893.0,1.0
7,"MOSCATELLI, DAVID A",221198.0,1,221198.0,1.0,221198.0,1.0
8,"POSSEMATO, RICHARD LEWIS",221198.0,2,470197.0,2.0,470197.0,1.0
9,"SKOK, JANE AMANDA",220825.0,2,745991.0,1.0,220825.0,1.0


### What Is the Most Common CI for each PI? 

In [42]:
# Calculate how many projects each PI has with each CI
temp = df_NYU3.groupby(['PI_NAME', 'IC_NAME'])['APPLICATION_ID'].nunique().sort_values(ascending=False)

# Convert into a dataframe and reset index
temp = temp.to_frame().reset_index()

# Rename APPLICATION_ID Count
temp.rename(columns={'APPLICATION_ID':'Count'}, inplace = True)

# Group by PI_NAME and select the resultant rows with the max value in the (numerical column -> Count)
# Sorted in ascending order of Count
temp = temp.groupby(['PI_NAME'],sort=True)[['IC_NAME', 'Count']].max()

# Rename IC_NAME Most_common_IC
temp.rename(columns={'IC_NAME':'Most_common_IC_2015', 'Count': 'Most_common_IC_Count_2015'}, inplace = True)
temp = temp.reset_index()

# View the first 3 observations
temp.head(3)

,PI_NAME,Most_common_IC_2015,Most_common_IC_Count_2015
0,"BINKA, FRED NEWTON",FOGARTY INTERNATIONAL CENTER,1
1,"BRAITHWAITE, RONALD SCOTT",NATIONAL INSTITUTE OF BIOMEDICAL IMAGING AND BIOENGINEERING,1
2,"CHANDARANA, HERSH",NATIONAL INSTITUTE OF BIOMEDICAL IMAGING AND BIOENGINEERING,1


In [43]:
# Merge the temp dataframe into the DF_2015 data frame 
# Use an outer merge
DF_2015= pd.merge(DF_2015, temp, on='PI_NAME', how = 'outer')

# Look at how many rows and columns there are
DF_2015.shape

(508, 9)

In [44]:
# View the first 3 observations
DF_2015.head(3)

,PI_NAME,NCI_New_Total_Cost_2015,Total_Projects_2015,Total_Cost_All_2015,Total_NCI_Projects_2015,NCI_Total_Cost_2015,Total_New_NCI_Projects_2015,Most_common_IC_2015,Most_common_IC_Count_2015
0,"ZELENIUCH-JAQUOTTE, ANNE",788279.0,1,788279.0,1.0,788279.0,1.0,NATIONAL CANCER INSTITUTE,1
1,"KIRCHHOFF, TOMAS",478926.0,2,663257.0,2.0,663257.0,1.0,NATIONAL CANCER INSTITUTE,2
2,"AIFANTIS, IANNIS",457533.0,4,1535056.0,4.0,1535056.0,1.0,NATIONAL CANCER INSTITUTE,4


# Link Data on PI Name <a class="anchor" id="link_name"></a>
Now we will merge together the `DF_2015` Date we just created with the pre processed `DF_2017` we read in at the beginning of this notebook. We will use an `outer` joint, so that we capture PI's the had grant(s) in only 2015, both 2015 and 2017, and only 2017

In [45]:
# Create a new dataframe by using an outer merge to merge the DF_2015 and DF_2017 dataframes
# merging on PI_NAME
DF = pd.merge(DF_2015, DF_2017, on='PI_NAME', how = 'outer')
DF.shape

(758, 17)

In [46]:
# Look at the first 5 observations
DF.head(5)

,PI_NAME,NCI_New_Total_Cost_2015,Total_Projects_2015,Total_Cost_All_2015,Total_NCI_Projects_2015,NCI_Total_Cost_2015,Total_New_NCI_Projects_2015,Most_common_IC_2015,Most_common_IC_Count_2015,Total_Projects_2017,Total_Cost_All_2017,Total_NCI_Projects_2017,NCI_Total_Cost_2017,Total_New_NCI_Projects_2017,NCI_New_Total_Cost_2017,Most_common_IC_2017,Most_common_IC_Count_2017
0,"ZELENIUCH-JAQUOTTE, ANNE",788279.0,1.0,788279.0,1.0,788279.0,1.0,NATIONAL CANCER INSTITUTE,1.0,1.0,694206.0,1.0,694206.0,NaN,NaN,NATIONAL CANCER INSTITUTE,1.0
1,"KIRCHHOFF, TOMAS",478926.0,2.0,663257.0,2.0,663257.0,1.0,NATIONAL CANCER INSTITUTE,2.0,1.0,484247.0,1.0,484247.0,NaN,NaN,NATIONAL CANCER INSTITUTE,1.0
2,"AIFANTIS, IANNIS",457533.0,4.0,1535056.0,4.0,1535056.0,1.0,NATIONAL CANCER INSTITUTE,4.0,6.0,2654279.0,6.0,2654279.0,1.0,536993.0,NATIONAL CANCER INSTITUTE,6.0
3,"CARUCCI, JOHN A",444422.0,1.0,444422.0,1.0,444422.0,1.0,NATIONAL CANCER INSTITUTE,1.0,1.0,206473.0,1.0,206473.0,NaN,NaN,NATIONAL CANCER INSTITUTE,1.0
4,"GOLD, HEATHER TAFFET",424833.0,1.0,424833.0,1.0,424833.0,1.0,NATIONAL CANCER INSTITUTE,1.0,1.0,406860.0,1.0,406860.0,NaN,NaN,NATIONAL CANCER INSTITUTE,1.0


In [47]:
# Look at the last 5 observations 
DF.tail(5)

,PI_NAME,NCI_New_Total_Cost_2015,Total_Projects_2015,Total_Cost_All_2015,Total_NCI_Projects_2015,NCI_Total_Cost_2015,Total_New_NCI_Projects_2015,Most_common_IC_2015,Most_common_IC_Count_2015,Total_Projects_2017,Total_Cost_All_2017,Total_NCI_Projects_2017,NCI_Total_Cost_2017,Total_New_NCI_Projects_2017,NCI_New_Total_Cost_2017,Most_common_IC_2017,Most_common_IC_Count_2017
753,"FALKNER, ANNEGRET LEA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,102740.0,NaN,NaN,NaN,NaN,NATIONAL INSTITUTE OF MENTAL HEALTH,1.0
754,"DOUTHIT, JESSICA P",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,34787.0,NaN,NaN,NaN,NaN,NATIONAL EYE INSTITUTE,1.0
755,"DORAN, KELLY",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,200674.0,NaN,NaN,NaN,NaN,NATIONAL INSTITUTE ON DRUG ABUSE,1.0
756,"DODSON, JOHN A",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,173824.0,NaN,NaN,NaN,NaN,NATIONAL INSTITUTE ON AGING,1.0
757,"ZOTTER, BRENDAN CARL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,35035.0,NaN,NaN,NaN,NaN,NATIONAL INSTITUTE OF NEUROLOGICAL DISORDERS AND STROKE,1.0


#### Fill in the Missing values with 0
Caution! it is often not the case that you should fill missing values with Zero. It is appropriate in this case because we know for example, FALKNER ANNEGRET LEA, was not a PI on an NIH project for NYU in 2015. Note that this narrows the scope of the question, though, because FALKNER ANNEGRET LEA could have worked on non-NIH projects in 2015, or she could have worked on NIH projects at a different institution in 2015. You have to be very mindful of what assumptions you are making and how you are narrowing the scope of your research when you fill in missing values with zero for your own project. 

In [48]:
# use the fillma function to fill all of the missing values with Zero
DF= DF.fillna(0)

# view the first 5 observations 
DF.head()

,PI_NAME,NCI_New_Total_Cost_2015,Total_Projects_2015,Total_Cost_All_2015,Total_NCI_Projects_2015,NCI_Total_Cost_2015,Total_New_NCI_Projects_2015,Most_common_IC_2015,Most_common_IC_Count_2015,Total_Projects_2017,Total_Cost_All_2017,Total_NCI_Projects_2017,NCI_Total_Cost_2017,Total_New_NCI_Projects_2017,NCI_New_Total_Cost_2017,Most_common_IC_2017,Most_common_IC_Count_2017
0,"ZELENIUCH-JAQUOTTE, ANNE",788279.0,1.0,788279.0,1.0,788279.0,1.0,NATIONAL CANCER INSTITUTE,1.0,1.0,694206.0,1.0,694206.0,0.0,0.0,NATIONAL CANCER INSTITUTE,1.0
1,"KIRCHHOFF, TOMAS",478926.0,2.0,663257.0,2.0,663257.0,1.0,NATIONAL CANCER INSTITUTE,2.0,1.0,484247.0,1.0,484247.0,0.0,0.0,NATIONAL CANCER INSTITUTE,1.0
2,"AIFANTIS, IANNIS",457533.0,4.0,1535056.0,4.0,1535056.0,1.0,NATIONAL CANCER INSTITUTE,4.0,6.0,2654279.0,6.0,2654279.0,1.0,536993.0,NATIONAL CANCER INSTITUTE,6.0
3,"CARUCCI, JOHN A",444422.0,1.0,444422.0,1.0,444422.0,1.0,NATIONAL CANCER INSTITUTE,1.0,1.0,206473.0,1.0,206473.0,0.0,0.0,NATIONAL CANCER INSTITUTE,1.0
4,"GOLD, HEATHER TAFFET",424833.0,1.0,424833.0,1.0,424833.0,1.0,NATIONAL CANCER INSTITUTE,1.0,1.0,406860.0,1.0,406860.0,0.0,0.0,NATIONAL CANCER INSTITUTE,1.0


## Develop measures for change over time  <a class="anchor" id="change_measures"></a>
Now that we have brought together the 2015 and 2017 data, we can create measures for the change in outcomes. 
We will develop two measures
1. Change in the number of NCI Projects
2. Change in the Total cost of NCI Projects

#### Change in the Number of National Cancer Institute Projects

In [49]:
# Create a new column (Change_in_NCI_Projects) that is equal to the difference in Total_NCI_Projects_2017 and Total_NCI_Projects_2015'
DF['Change_in_NCI_Projects'] = DF['Total_NCI_Projects_2017'] - DF['Total_NCI_Projects_2015']

# Look at the first 5 observations for select variables 
DF[['PI_NAME','Total_NCI_Projects_2017', 'Total_NCI_Projects_2015', 'Change_in_NCI_Projects']].head()

,PI_NAME,Total_NCI_Projects_2017,Total_NCI_Projects_2015,Change_in_NCI_Projects
0,"ZELENIUCH-JAQUOTTE, ANNE",1.0,1.0,0.0
1,"KIRCHHOFF, TOMAS",1.0,2.0,-1.0
2,"AIFANTIS, IANNIS",6.0,4.0,2.0
3,"CARUCCI, JOHN A",1.0,1.0,0.0
4,"GOLD, HEATHER TAFFET",1.0,1.0,0.0


#### Change in the Cost of National Cancer Institute Projects

In [50]:
# Create a new column (Change_in_NCI_Cost) that is equal to the difference in NCI_Total_Cost_2017 and NCI_Total_Cost_2015'
DF['Change_in_NCI_Cost'] = DF['NCI_Total_Cost_2017'] - DF['NCI_Total_Cost_2015']

# Look at the first 5 observations for select variables 
DF[['PI_NAME','NCI_Total_Cost_2017', 'NCI_Total_Cost_2015', 'Change_in_NCI_Cost']].head()


,PI_NAME,NCI_Total_Cost_2017,NCI_Total_Cost_2015,Change_in_NCI_Cost
0,"ZELENIUCH-JAQUOTTE, ANNE",694206.0,788279.0,-94073.0
1,"KIRCHHOFF, TOMAS",484247.0,663257.0,-179010.0
2,"AIFANTIS, IANNIS",2654279.0,1535056.0,1119223.0
3,"CARUCCI, JOHN A",206473.0,444422.0,-237949.0
4,"GOLD, HEATHER TAFFET",406860.0,424833.0,-17973.0


# Results <a class="anchor" id="results"></a>
Now that the data is prepaired, we have answer some questions about PIs in 2015 and 2017

In [51]:
DF.columns

Index(['PI_NAME', 'NCI_New_Total_Cost_2015', 'Total_Projects_2015',
       'Total_Cost_All_2015', 'Total_NCI_Projects_2015', 'NCI_Total_Cost_2015',
       'Total_New_NCI_Projects_2015', 'Most_common_IC_2015',
       'Most_common_IC_Count_2015', 'Total_Projects_2017',
       'Total_Cost_All_2017', 'Total_NCI_Projects_2017', 'NCI_Total_Cost_2017',
       'Total_New_NCI_Projects_2017', 'NCI_New_Total_Cost_2017',
       'Most_common_IC_2017', 'Most_common_IC_Count_2017',
       'Change_in_NCI_Projects', 'Change_in_NCI_Cost'],
      dtype='object')

#### How many PIs form NYU were there in 2015?

In [52]:
# Look at the first 2 observations for select variables 
DF.head(2)

,PI_NAME,NCI_New_Total_Cost_2015,Total_Projects_2015,Total_Cost_All_2015,Total_NCI_Projects_2015,NCI_Total_Cost_2015,Total_New_NCI_Projects_2015,Most_common_IC_2015,Most_common_IC_Count_2015,Total_Projects_2017,Total_Cost_All_2017,Total_NCI_Projects_2017,NCI_Total_Cost_2017,Total_New_NCI_Projects_2017,NCI_New_Total_Cost_2017,Most_common_IC_2017,Most_common_IC_Count_2017,Change_in_NCI_Projects,Change_in_NCI_Cost
0,"ZELENIUCH-JAQUOTTE, ANNE",788279.0,1.0,788279.0,1.0,788279.0,1.0,NATIONAL CANCER INSTITUTE,1.0,1.0,694206.0,1.0,694206.0,0.0,0.0,NATIONAL CANCER INSTITUTE,1.0,0.0,-94073.0
1,"KIRCHHOFF, TOMAS",478926.0,2.0,663257.0,2.0,663257.0,1.0,NATIONAL CANCER INSTITUTE,2.0,1.0,484247.0,1.0,484247.0,0.0,0.0,NATIONAL CANCER INSTITUTE,1.0,-1.0,-179010.0


In [53]:
# Subset dataframe for PIs who had more than zero total projects in 2015
temp = DF[DF['Total_Projects_2015'] > 0] 

# Count the number of unique PI names
temp["PI_NAME"].nunique()

508

#### How many PIs form NYU were there in 2017?

In [54]:
# Subset dataframe for PIs who had more than zero total projects in 2017
temp = DF[DF['Total_Projects_2017'] > 0] 

# Count the number of unique PI names
temp["PI_NAME"].nunique()

590

#### How many PIs from NYU had NCI projects in 2015?

In [55]:
# Subset dataframe for PIs who had more than zero NCI projects in 2015
temp = DF[DF['Total_NCI_Projects_2015'] > 0] 

# Count the number of unique PI names
temp["PI_NAME"].nunique()

68

#### How many PIs from NYU had NCI projects in 2017?

In [56]:
# Subset dataframe for PIs who had more than zero NCI projects in 2017
temp = DF[DF['Total_NCI_Projects_2017'] > 0] 

# Count the number of unique PI names
temp["PI_NAME"].nunique()

80

#### How many PIs from NYU had new NCI projects in 2015?

In [57]:
# Subset dataframe for PIs who had more than zero new NCI projects in 2015
temp = DF[DF['Total_New_NCI_Projects_2015'] > 0] 

# Count the number of unique PI names
temp["PI_NAME"].nunique()

14

#### How many PIs from NYU had new NCI projects in 2017?

In [58]:
# Subset dataframe for PIs who had more than zero new NCI projects in 2017
temp = DF[DF['Total_New_NCI_Projects_2017'] > 0] 

# Count the number of unique PI names
temp["PI_NAME"].nunique()

26

#### How many PIs only had non NCI Project(s) in 2015 but had at least one NCI project in 2017?

In [59]:
# Subset dataframe for PIs who had zero NCI projects in 2015
# and more than zero projects in 2015
# and more than zero NCI projects in 2017
Converted_PIs = DF[(DF['Total_NCI_Projects_2015'] == 0) & 
           (DF['Total_Projects_2015'] > 0) &
           (DF['Total_NCI_Projects_2017'] > 0)]

# Count the number of unique PI names
Converted_PIs["PI_NAME"].nunique()

2

In [60]:
Converted_PIs.head()

,PI_NAME,NCI_New_Total_Cost_2015,Total_Projects_2015,Total_Cost_All_2015,Total_NCI_Projects_2015,NCI_Total_Cost_2015,Total_New_NCI_Projects_2015,Most_common_IC_2015,Most_common_IC_Count_2015,Total_Projects_2017,Total_Cost_All_2017,Total_NCI_Projects_2017,NCI_Total_Cost_2017,Total_New_NCI_Projects_2017,NCI_New_Total_Cost_2017,Most_common_IC_2017,Most_common_IC_Count_2017,Change_in_NCI_Projects,Change_in_NCI_Cost
73,"DYNLACHT, BRIAN D",0.0,2.0,688410.0,0.0,0.0,0.0,NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES,1.0,4.0,1152873.0,1.0,196577.0,1.0,196577.0,NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES,2.0,1.0,196577.0
284,"SCHMIDT, BRIAN L",0.0,1.0,396250.0,0.0,0.0,0.0,NATIONAL INSTITUTE OF DENTAL & CRANIOFACIAL RESEARCH,1.0,3.0,1100085.0,1.0,587089.0,0.0,0.0,NATIONAL INSTITUTE OF DENTAL & CRANIOFACIAL RESEARCH,2.0,1.0,587089.0


In [61]:
# look at the first 5 observations for select variables
Converted_PIs[['PI_NAME', 'Total_Projects_2015', 'Total_Cost_All_2015', 'Total_NCI_Projects_2015', 
      'Total_NCI_Projects_2017', 'NCI_Total_Cost_2017', 'Most_common_IC_2015']].head()

,PI_NAME,Total_Projects_2015,Total_Cost_All_2015,Total_NCI_Projects_2015,Total_NCI_Projects_2017,NCI_Total_Cost_2017,Most_common_IC_2015
73,"DYNLACHT, BRIAN D",2.0,688410.0,0.0,1.0,196577.0,NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES
284,"SCHMIDT, BRIAN L",1.0,396250.0,0.0,1.0,587089.0,NATIONAL INSTITUTE OF DENTAL & CRANIOFACIAL RESEARCH


# Checkpoint 2 (Assignment 3 due March 10th) <a class="anchor" id="checkpoint2"></a>

How many PIs had a NCI funded projects in 2017 but not 2015? (1point)

For those who have at least one NCI funded project in 2015, how many NCI funded projects do they have on average? (1point)

For those who have at least one NCI funded project in 2017, how many NCI funded projects do they have on average? (1point)

For the PIs who only had non-NCI Project(s) in 2015 but had at least one NCI project in 2017, what are the 5 most common ICs they are moving from (in total, not per PI)? (2points)

Can we sum the `Total_Cost_All_2015` column in the `DF` dataframe to get the total cost of projects in 2015? Why or why not? (2points)
- This is not a question you need code to answer, instead, create a Markdown cell, and answer in a complete sentence. 

#### 1. How many PIs had a NCI funded projects in 2017 but not 2015? (1point)

In [62]:
Converted_PIs = DF[(DF['Total_NCI_Projects_2017'] > 0) & 
           (DF['Total_NCI_Projects_2015'] == 0)]

# Count the number of unique PI names
Converted_PIs["PI_NAME"].nunique()

41

#### 2. For those who have at least one NCI funded project in 2015, how many NCI funded projects do they have on average? (1point)

In [63]:
# Subset dataframe for PIs who had more than zero NCI projects in 2015
temp = DF[DF['Total_NCI_Projects_2015'] > 0] 

temp["PI_NAME"].nunique()
temp

temp2 = temp ["Total_NCI_Projects_2015"].mean()
temp2

1.75

#### 3. For those who have at least one NCI funded project in 2017, how many NCI funded projects do they have on average? (1point)

In [64]:
# Subset dataframe for PIs who had more than zero NCI projects in 2017
temp = DF[DF['Total_NCI_Projects_2017'] > 0] 
temp
temp3=temp["Total_NCI_Projects_2017"].mean()
temp3

1.75

#### 4. For the PIs who only had non-NCI Project(s) in 2015 but had at least one NCI project in 2017, what are the 5 most common ICs they are moving from (in total, not per PI)? (2points)

In [65]:
# Subset dataframe for PIs who had zero NCI projects in 2015
# and more than zero projects in 2015
# and more than zero NCI projects in 2017
Converted_PIs = DF[(DF['Total_NCI_Projects_2015'] == 0) & 
           (DF['Total_Projects_2015'] > 0) &
           (DF['Total_NCI_Projects_2017'] > 0)]
Converted_PIs

,PI_NAME,NCI_New_Total_Cost_2015,Total_Projects_2015,Total_Cost_All_2015,Total_NCI_Projects_2015,NCI_Total_Cost_2015,Total_New_NCI_Projects_2015,Most_common_IC_2015,Most_common_IC_Count_2015,Total_Projects_2017,Total_Cost_All_2017,Total_NCI_Projects_2017,NCI_Total_Cost_2017,Total_New_NCI_Projects_2017,NCI_New_Total_Cost_2017,Most_common_IC_2017,Most_common_IC_Count_2017,Change_in_NCI_Projects,Change_in_NCI_Cost
73,"DYNLACHT, BRIAN D",0.0,2.0,688410.0,0.0,0.0,0.0,NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES,1.0,4.0,1152873.0,1.0,196577.0,1.0,196577.0,NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES,2.0,1.0,196577.0
284,"SCHMIDT, BRIAN L",0.0,1.0,396250.0,0.0,0.0,0.0,NATIONAL INSTITUTE OF DENTAL & CRANIOFACIAL RESEARCH,1.0,3.0,1100085.0,1.0,587089.0,0.0,0.0,NATIONAL INSTITUTE OF DENTAL & CRANIOFACIAL RESEARCH,2.0,1.0,587089.0


#### There are only 2 PI who only had non-NCI Project(s) in 2015 but had at least one NCI project in 2017. They are DYNLACHT, BRIAN D, and SCHMIDT, BRIAN L.

##### 5. Can we sum the `Total_Cost_All_2015` column in the `DF` dataframe to get the total cost of projects in 2015? Why or why not? (2points)
- This is not a question you need code to answer, instead, create a Markdown cell, and answer in a complete sentence. 

#### No, because each project can have more than one PI, but each PI has one value of Total_Costs_All_2015 in the DF dataframe which in fact equals to the total cost of each project. If we sum them up, we will overcalculate the total costs of projects in 2015.